In [ ]:
import pyodbc
import pandas as pd

# SQLデータベース接続関数
def sql_connection():
    connection_string = (
        "Driver={SQL Server};"
        "Server=MMD-LOGISTICS\SQLEXPRESS;"
        "Database=LogiRecoDB;"
        "UID=dev;"
        "PWD=awadmin;"
    )
    connection = pyodbc.connect(connection_string)
    connection.cursor().execute("SET NOCOUNT ON")
    return connection

# マスタ更新関数
def update_master(workbook_path):
    # Excelファイルを開く
    wb = pd.ExcelFile(workbook_path)
    pit_master = wb.parse("PITマスタ")
    base_cd = pit_master.at[2, 'B']  # B3セル
    
    # SQL実行
    conn = sql_connection()
    cursor = conn.cursor()
    cursor.execute(f"SELECT BASE_NM FROM TBM_BASE WHERE BASE_CD='{base_cd}'")
    result = cursor.fetchone()
    if result:
        pit_master.at[1, 'B'] = result[0]  # セル(2, 2)に書き込み

    # 稼働日を取得
    cursor.execute(f"SELECT MAX(LINE_DATE) FROM TBM_WORK WHERE BASE_CD='{base_cd}' AND LINE_DATE <= '{pit_master.at[2, 'D']}'")
    result = cursor.fetchone()
    if result:
        operation_day = result[0]
    else:
        print("マスタデータが存在しません")

    # 作業情報を取得
    sql = f"""SELECT TRIM(B.KOTEI_NM), TRIM(A.WORK_NM), A.KOTEI_ID, A.WORK_ID, A.TYOKU 
              FROM TBM_WORK A 
              INNER JOIN TBM_KOTEI B ON A.BASE_CD=B.BASE_CD AND A.KOTEI_ID=B.KOTEI_ID 
              WHERE A.BASE_CD='{base_cd}' AND A.LINE_DATE='{operation_day}' 
              ORDER BY KOTEI_ID, WORK_ID"""
    df = pd.read_sql(sql, conn)
    df.to_excel(workbook_path, sheet_name='PITマスタ', startrow=5, startcol=13, index=False)  # N6セルからデータを貼り付け

# 実績読み込み関数
def read_performance(workbook_path):
    # Excelファイルを開く
    wb = pd.ExcelFile(workbook_path)
    evaluation_sheet = wb.parse("評価シート")
    operation_day = evaluation_sheet.at[2, 'J']  # J3セル

    # SQL実行
    conn = sql_connection()
    sql = f"""SELECT KOTEI_ID, WORK_ID, JISEKI_DT, JISEKI_DT2, TYOKU 
              FROM TBD_JISEKI 
              WHERE BASE_CD='{base_cd}' AND LINE_DATE='{operation_day}' 
              ORDER BY SEQ"""
    df = pd.read_sql(sql, conn)
    df.to_excel(workbook_path, sheet_name='実績', startrow=1, startcol=1, index=False)  # B2セルからデータを貼り付け

# 評価実行関数
def evaluate(workbook_path):
    update_master(workbook_path)
    read_performance(workbook_path)

# 実行
workbook_path = 'path_to_your_workbook.xlsx'
evaluate(workbook_path)
